# Тестовое задание для RSRCH

Задание: https://docs.google.com/document/d/1V66EmFAEiYz36Gvqv3IySPEuoNzmdqbn_pbPv-g4K7g/edit?usp=sharing

In [2]:
import pandas as pd
import numpy as np
import pymorphy2
import scipy.sparse as sparse

morph = pymorphy2.MorphAnalyzer()

from lightfm import LightFM

Для выполнения задания будем использовать файл с трудовыми действиями (`LaborAction.csv`) и описание курсов `teach-in.ru`. Можно использовать и другие источники данных, при этом подход к выполнению задания останется прежним.

Импортируем данные для трудовых действий

In [3]:
df_lab_actions = pd.read_csv('LaborAction.csv', sep=';', header=None)
df_lab_actions

,0,1
0,Разработка и реализация программ учебных дисци...,a347061b8b252e3a09a78d81c6d9fb82
1,Осуществление профессиональной деятельности в ...,a347061b8b252e3a09a78d81c6d9fb82
2,Участие в разработке и реализации программы ра...,a347061b8b252e3a09a78d81c6d9fb82
3,Планирование и проведение учебных занятий,a347061b8b252e3a09a78d81c6d9fb82
4,Систематический анализ эффективности учебных з...,a347061b8b252e3a09a78d81c6d9fb82
...,...,...
131304,Исправление ошибок в программном коде модулей ...,d550e3377b1e2575b8f4045bc8703913
131305,Проведение тестирования программного обеспечен...,d2c2de43d3b48430a1b43967e0b68d85
131306,Сбор результатов тестирования программного обе...,d2c2de43d3b48430a1b43967e0b68d85
131307,Оценка соответствия результатов тестирования п...,d2c2de43d3b48430a1b43967e0b68d85


Создадим вспомогательную функцию, которая будет создавать из строк трудовых действий список ключевых слов - существительных в единственном числе именительном падаже 

In [4]:
def get_keys(value):
    keys = []
    for s in '1234567890-.,:;-?!\"': # удаляем не информативные символы из строки
        value = value.replace(s, ' ')
    for word in value.split(' '): # делим строку на слова
        p = morph.parse(word)[0] # для каждого слова выбираем разбор с максимальной вероятностью
        word_norm_form = p.normal_form # выделяем начальную форму слова
        if 'NOUN' in p.tag and word_norm_form not in keys: # если слово существительное и еще не было включено в список, включаем
            keys.append(word_norm_form)
    return keys

Также создадим аналогичную функцию, которая будет создавать список ключевых слов из серии трудовых действий

In [5]:
def get_keys_list(series):
    keys_list = []
    for value in series:
        for s in '1234567890-.,:;-?!\"':
            value = value.replace(s, ' ')
        for word in value.split(' '):
            p = morph.parse(word)[0]
            word_norm_form = p.normal_form
            if 'NOUN' in p.tag and word_norm_form not in keys_list:
                keys_list.append(word_norm_form)
    return keys_list

Создадим список ключевых слов из трудовых действий из `LaborAction.csv`

In [6]:
lab_actions_keys = get_keys_list(df_lab_actions[0])
lab_actions_keys

['разработка',
 'реализация',
 'программа',
 'дисциплина',
 'рамка',
 'осуществление',
 'деятельность',
 'соответствие',
 'требование',
 'стандарт',
 'среднее',
 'образование',
 'участие',
 'развитие',
 'организация',
 'цель',
 'создание',
 'среда',
 'планирование',
 'проведение',
 'занятие',
 'анализ',
 'эффективность',
 'подход',
 'обучение',
 'контроль',
 'оценка',
 'достижение',
 'результат',
 'освоение',
 'формирование',
 'действие',
 'навык',
 'технология',
 'мотивация',
 'знание',
 'основа',
 'тестирование',
 'метод',
 'возможность',
 'ребёнок',
 'регулирование',
 'поведение',
 'обеспечение',
 'число',
 'форма',
 'работа',
 'постановка',
 'способность',
 'характер',
 'определение',
 'принятие',
 'правило',
 'устав',
 'распорядок',
 'проектирование',
 'вид',
 'ситуация',
 'событие',
 'сфера',
 '(культура',
 'переживание',
 'ориентация',
 'помощь',
 'поддержка',
 'орган',
 'самоуправление',
 'поддержание',
 'уклад',
 'атмосфера',
 'традиция',
 'жизнь',
 'активность',
 'самостоятел

Импортируем данные о курсах `teach-in.ru`

In [7]:
teachin_df = pd.read_csv('teach_in.csv', sep=';')
teachin_df.head()

,Ссылка на курс,Количество лекций,Название курса,Категория,Количество просмотров,Автор,Ссылка на автора,Видеолекции,О курсе
0,https://teach-in.ru//course/one-hundred-hours-...,29 лекций,100 часов школьной биологии. Анатомия и физиол...,Биология,1.0,Дубынин Вячеслав Альбертович,https://teach-in.ru//lecturer/dybinin,/lecture/2021-07-30-Dubynin Вступительное слов...,",Список всех тем лекций, , Вступительное слово..."
1,https://teach-in.ru//course/one-hundred-hours-...,11 лекций,100 часов школьной биологии. Общая биология,Биология,NaN,Дубынин Вячеслав Альбертович,https://teach-in.ru//lecturer/dybinin,/lecture/2021-10-11-Dubynin Лекция 1. Вода и м...,",Список всех тем лекций,Лекция 1. Вода и минер..."
2,https://teach-in.ru//course/basics-of-radioche...,12 лекций,Basics of radiochemistry. Лекции,Химия,NaN,Петров Владимир Геннадьевич,https://teach-in.ru//lecturer/petrov-v-g,/lecture/2021-09-10-Petrov-2 Лекция 1. Introdu...,",Список всех тем лекций,Лекция 1. Introduction..."
3,https://teach-in.ru//course/basics-of-radioche...,10 лекций,Basics of radiochemistry. Семинары,Химия,NaN,Петров Владимир Геннадьевич,https://teach-in.ru//lecturer/petrov-v-g,/lecture/2021-09-10-Petrov-1 Семинар 1. Types ...,",Список всех тем лекций,Семинар 1. Types of ra..."
4,https://teach-in.ru//course/entagled-diffury,4 лекции,ENTANGLED Дифференциальные уравнения,Математика,NaN,Гайер Алексей Вячеславович,https://teach-in.ru//lecturer/gaier-av,/lecture/2021-12-21-Gaier Семинар 1. Дифференц...,",Список всех тем лекций,Семинар 1. Дифференциа..."


Оставим только те данные, которые нам понадобятся в будущем

In [8]:
teachin_df = teachin_df[['Название курса', 'Ссылка на курс', 'О курсе']]
teachin_df

,Название курса,Ссылка на курс,О курсе
0,100 часов школьной биологии. Анатомия и физиол...,https://teach-in.ru//course/one-hundred-hours-...,",Список всех тем лекций, , Вступительное слово..."
1,100 часов школьной биологии. Общая биология,https://teach-in.ru//course/one-hundred-hours-...,",Список всех тем лекций,Лекция 1. Вода и минер..."
2,Basics of radiochemistry. Лекции,https://teach-in.ru//course/basics-of-radioche...,",Список всех тем лекций,Лекция 1. Introduction..."
3,Basics of radiochemistry. Семинары,https://teach-in.ru//course/basics-of-radioche...,",Список всех тем лекций,Семинар 1. Types of ra..."
4,ENTANGLED Дифференциальные уравнения,https://teach-in.ru//course/entagled-diffury,",Список всех тем лекций,Семинар 1. Дифференциа..."
...,...,...,...
538,Элементы теории чисел. Семинары,https://teach-in.ru//course/elements-of-number...,",Семинары по базовому курсу теории чисел на 1 ..."
539,Элементы топологии и симплектической геометрии...,https://teach-in.ru//course/elements-of-topolo...,",Спецкурс для студентов 1–6 курсов,Список всех..."
540,Элементы топологии и симплектической геометрии...,https://teach-in.ru//course/elements-of-topolo...,",Спецкурс для студентов 1–6 курсов,Список всех..."
541,Эллиптические операторы и теорема об индексе,https://teach-in.ru//course/elliptic-operators...,",Основная цель курса – познакомить слушателей ..."


Создадим список ключевых слов для каждого курса

In [9]:
teachin_df['course_keys'] = teachin_df['О курсе'].apply(get_keys)
teachin_df

,Название курса,Ссылка на курс,О курсе,course_keys
0,100 часов школьной биологии. Анатомия и физиол...,https://teach-in.ru//course/one-hundred-hours-...,",Список всех тем лекций, , Вступительное слово...","[список, лекция, слово, курс, сто, час, биолог..."
1,100 часов школьной биологии. Общая биология,https://teach-in.ru//course/one-hundred-hours-...,",Список всех тем лекций,Лекция 1. Вода и минер...","[список, лекция, вода, соль, соединение, принц..."
2,Basics of radiochemistry. Лекции,https://teach-in.ru//course/basics-of-radioche...,",Список всех тем лекций,Лекция 1. Introduction...","[список, лекция]"
3,Basics of radiochemistry. Семинары,https://teach-in.ru//course/basics-of-radioche...,",Список всех тем лекций,Семинар 1. Types of ra...","[список, лекция, семинар]"
4,ENTANGLED Дифференциальные уравнения,https://teach-in.ru//course/entagled-diffury,",Список всех тем лекций,Семинар 1. Дифференциа...","[список, лекция, семинар, уравнение, го, поряд..."
...,...,...,...,...
538,Элементы теории чисел. Семинары,https://teach-in.ru//course/elements-of-number...,",Семинары по базовому курсу теории чисел на 1 ...","[семинар, курс, теория, число, список, лекция,..."
539,Элементы топологии и симплектической геометрии...,https://teach-in.ru//course/elements-of-topolo...,",Спецкурс для студентов 1–6 курсов,Список всех...","[спецкурс, студент, курс, список, лекция, крив..."
540,Элементы топологии и симплектической геометрии...,https://teach-in.ru//course/elements-of-topolo...,",Спецкурс для студентов 1–6 курсов,Список всех...","[спецкурс, студент, курс, список, лекция, теор..."
541,Эллиптические операторы и теорема об индексе,https://teach-in.ru//course/elliptic-operators...,",Основная цель курса – познакомить слушателей ...","[цель, слушателей с, понятие, метод, теория, ф..."


Создадим датафрейм, который будем использовать в рекомендательной системе. Объектами будут ключевые слова, полученные из трудовых действий. Им будем рекомендовать курсы `teach-in.ru`

In [10]:
df = pd.DataFrame(index=lab_actions_keys, columns=teachin_df['Название курса'])
df

Название курса,100 часов школьной биологии. Анатомия и физиология человека,100 часов школьной биологии. Общая биология,Basics of radiochemistry. Лекции,Basics of radiochemistry. Семинары,ENTANGLED Дифференциальные уравнения,ENTANGLED Квантовая теория,ENTANGLED Математический анализ. 2 семестр,Python и облачные вычисления в науке,Автоморфизмы аффинных алгебраических многообразий,Алгебра. Семинары. Часть 1,...,Электромагнетизм. Семинары,Элементы строения вещества,Элементы теории гомологий,Элементы теории чисел,Элементы теории чисел,Элементы теории чисел. Семинары,Элементы топологии и симплектической геометрии. Часть 1,Элементы топологии и симплектической геометрии. Часть 2,Эллиптические операторы и теорема об индексе,Эмбриология
разработка,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
реализация,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
программа,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
дисциплина,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
рамка,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
линь,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
динамик,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
патч,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
клеммов,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
for key in lab_actions_keys:
    vals = []
    interaction_list = []
    for course in teachin_df['Название курса']:
        vals.append(key in teachin_df.loc[teachin_df['Название курса']==course,'course_keys'].values[0])
    df.loc[key] = vals

In [12]:
df = df.astype(int)
df

Название курса,100 часов школьной биологии. Анатомия и физиология человека,100 часов школьной биологии. Общая биология,Basics of radiochemistry. Лекции,Basics of radiochemistry. Семинары,ENTANGLED Дифференциальные уравнения,ENTANGLED Квантовая теория,ENTANGLED Математический анализ. 2 семестр,Python и облачные вычисления в науке,Автоморфизмы аффинных алгебраических многообразий,Алгебра. Семинары. Часть 1,...,Электромагнетизм. Семинары,Элементы строения вещества,Элементы теории гомологий,Элементы теории чисел,Элементы теории чисел,Элементы теории чисел. Семинары,Элементы топологии и симплектической геометрии. Часть 1,Элементы топологии и симплектической геометрии. Часть 2,Эллиптические операторы и теорема об индексе,Эмбриология
разработка,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
реализация,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
программа,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
дисциплина,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
рамка,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
линь,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
динамик,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
патч,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
клеммов,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
df = pd.read_csv('df.csv', index_col=0)
df

,100 часов школьной биологии. Анатомия и физиология человека,100 часов школьной биологии. Общая биология,Basics of radiochemistry. Лекции,Basics of radiochemistry. Семинары,ENTANGLED Дифференциальные уравнения,ENTANGLED Квантовая теория,ENTANGLED Математический анализ. 2 семестр,Python и облачные вычисления в науке,Автоморфизмы аффинных алгебраических многообразий,Алгебра. Семинары. Часть 1,...,Электромагнетизм. Семинары,Элементы строения вещества,Элементы теории гомологий,Элементы теории чисел,Элементы теории чисел.1,Элементы теории чисел. Семинары,Элементы топологии и симплектической геометрии. Часть 1,Элементы топологии и симплектической геометрии. Часть 2,Эллиптические операторы и теорема об индексе,Эмбриология
разработка,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
реализация,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
программа,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,1,0,0,0,0,0
дисциплина,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
рамка,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
линь,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
динамик,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
патч,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
клеммов,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Обучим модель рекомендательной системы. Будем использовать библиотеку <a href='https://github.com/lyst/lightfm'>LightFM</a>

In [15]:
model = LightFM(loss='warp-kos')
model.fit(sparse.coo_matrix(df))

Получим 3 лучшие рекомендации для объекта.

In [41]:
item = 20 # индекс объекта, для которого получаем рекомендации
preds = model.predict(item, list(range(len(df.columns))))
preds = pd.Series(preds).sort_values()
preds = preds ** 2
preds = ((preds-min(preds))*100/(max(preds)-min(preds))).head(3) # берем только 3 лучшие рекомендации
preds

381    100.000000
331     97.257538
117     96.880913
dtype: float32

In [42]:
print('Ключевое слово:', df.index[item])
print('Рекомедованные курсы:')
result_df = pd.DataFrame(columns=['№ п/п', 'Курсы', 'Балл', 'Ссылка'])
for i, course in enumerate(zip(df.columns[preds.index], preds.values)):
    result_df.loc[len(result_df)] = [i+1, course[0], int(course[1]), teachin_df.iloc[preds.index[i], 1]]
result_df

Ключевое слово: занятие
Рекомедованные курсы:


,№ п/п,Курсы,Балл,Ссылка
0,1,Решение прикладных задач с помощью программиро...,100,https://teach-in.ru//course/solving-applied-pr...
1,2,Общая физика для школьников. Механика,97,https://teach-in.ru//course/general-physics-fo...
2,3,Дифференциальная геометрия и топология.2,96,https://teach-in.ru//course/differential-geome...
